In [1]:
import re
import datetime
import pandas as pd
import json
from osisoft.pidevclub.piwebapi.pi_web_api_client import PIWebApiClient
from pandasgui import show
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

In [2]:
client = PIWebApiClient("https://piwebapi.cammesa.com/piwebapi",
                            useKerberos=False, verifySsl=True)

In [3]:
verano1_start = datetime.date(2021, 12, 1)
verano1_end = datetime.date(2022, 3, 1)

verano2_start = datetime.date(2022, 12, 1)
verano2_end = datetime.date(2023, 3, 1)

verano3_start = datetime.date(2023, 12, 1)
verano3_end = datetime.date(2024, 2, 28)

In [22]:
periodos = [
    #(verano1_start, verano1_end),
    #(verano2_start, verano2_end),
    (verano3_start, verano3_end)
]

In [23]:
df_paths = pd.read_csv('paths_reducido.csv')
paths = df_paths['Paths'].tolist()

In [24]:
from osisoft.pidevclub.piwebapi.rest import ApiException
df_nea = pd.DataFrame()

for periodo in periodos:
    start_datetime = datetime.datetime.combine(periodo[0], datetime.time())
    end_datetime = datetime.datetime.combine(periodo[1], datetime.time())

    for k, pt in enumerate(paths):
        try:
            df_interpolate = client.data.get_interpolated_values(
                path=pt,
                start_time=start_datetime,
                end_time=end_datetime,
                interval="1m"
            )

            if df_interpolate.empty or not all(col in df_interpolate.columns for col in ['Timestamp', 'Value']):
                print(f"Tag not found or data is missing for path: {pt}. Skipping...")
                continue 

            df_interpolate = df_interpolate[['Timestamp', 'Value']]
            df_interpolate.set_index('Timestamp', inplace=True)

            column_name = pt.replace('pi:\\\\CAMHIS01\\', '')

            df_interpolate.rename(columns={'Value': column_name}, inplace=True)

            if df_nea.empty:
                df_nea = df_interpolate
            else:
                df_nea = pd.concat([df_nea, df_interpolate], axis=1)

        except ApiException as e:
            if e.status == 404:
                print(f"Tag not found for path: {pt}. Skipping...")
                continue  # Skip to the next iteration
            else:
                # Handle other ApiException or re-raise the exception if needed
                raise




In [25]:
df_nea.head()

,DEM_NEA/TOT_DEM_____P.AV,RESISTEN/TEMP.AV,RINCON/TEMP.AV,A.BROWN/132/ROCA1/I.AV,A.BROWN/132/ROCA1/P.AV,A.BROWN/132/ROCA1/Q.AV,ALLEN/132/G.ROCA/I.AV,ALLEN/132/G.ROCA/P.AV,ALLEN/132/G.ROCA/Q.AV,ALLEN/132/G.ROCA/V.AV,...,S.ISIDRO/500/TRAFO2/V.AV,SGO.ESTE/13/ROCA/I.AV,SMARTNEA/13/TRAFO2/I.AV,SMARTNEA/13/TRAFO2/P.AV,SMARTNEA/13/TRAFO2/Q.AV,SMARTNEA/13/TRAFO2/V.AV,SMARTNEA/132/E.COLOR/I.AV,SMARTNEA/132/E.COLOR/P.AV,SMARTNEA/132/E.COLOR/Q.AV,SMARTNEA/132/E.COLOR/V.AV
Timestamp,,,,,,,,,,,,,,,,,,,,,
2023-12-01T03:00:00Z,1824.61365,26.125362,25.588234,218.200623,41.742195,-25.824749,116.700000,-25.259998,-8.980000,130.780000,...,497.503540,170.417725,109.800000,-2.47698,-0.78408,13.6092,0.0,0.0,0.0,133.6
2023-12-01T03:01:00Z,1809.75700,26.125362,25.588234,219.167450,42.049820,-25.824749,115.200000,-24.930000,-9.029999,130.829987,...,497.503540,165.913910,109.800000,-2.47698,-0.78408,13.6092,0.0,0.0,0.0,133.6
2023-12-01T03:02:00Z,1820.62354,26.125362,25.588234,222.390200,42.027847,-26.044480,114.500000,-24.680000,-9.029999,130.970000,...,496.771057,165.163269,109.800000,-2.47698,-0.78408,13.6092,0.0,0.0,0.0,133.6
2023-12-01T03:03:00Z,1813.32153,26.125362,25.588234,225.693512,41.522470,-25.517128,112.599991,-24.309998,-8.580000,130.989990,...,496.771057,165.163269,111.599991,-2.47698,-0.78408,13.6092,0.0,0.0,0.0,133.6
2023-12-01T03:04:00Z,1805.46729,26.125362,25.588234,240.034790,44.510796,-25.121613,114.000000,-24.610000,-8.330000,130.989990,...,496.771057,168.916443,111.599991,-2.47698,-0.78408,13.6092,0.0,0.0,0.0,133.6


In [26]:
df_nea_hora = df_nea.copy()
df_nea_hora.index = pd.to_datetime(df_nea_hora.index).tz_convert('America/Argentina/Buenos_Aires').tz_localize(None)

df_nea_hora

,DEM_NEA/TOT_DEM_____P.AV,RESISTEN/TEMP.AV,RINCON/TEMP.AV,A.BROWN/132/ROCA1/I.AV,A.BROWN/132/ROCA1/P.AV,A.BROWN/132/ROCA1/Q.AV,ALLEN/132/G.ROCA/I.AV,ALLEN/132/G.ROCA/P.AV,ALLEN/132/G.ROCA/Q.AV,ALLEN/132/G.ROCA/V.AV,...,S.ISIDRO/500/TRAFO2/V.AV,SGO.ESTE/13/ROCA/I.AV,SMARTNEA/13/TRAFO2/I.AV,SMARTNEA/13/TRAFO2/P.AV,SMARTNEA/13/TRAFO2/Q.AV,SMARTNEA/13/TRAFO2/V.AV,SMARTNEA/132/E.COLOR/I.AV,SMARTNEA/132/E.COLOR/P.AV,SMARTNEA/132/E.COLOR/Q.AV,SMARTNEA/132/E.COLOR/V.AV
Timestamp,,,,,,,,,,,,,,,,,,,,,
2023-12-01 00:00:00,1824.61365,26.125362,25.588234,218.200623,41.742195,-25.824749,116.700000,-25.259998,-8.980000,130.780000,...,497.503540,170.417725,109.800000,-2.47698,-0.78408,13.609200,0.0,0.0,0.0,133.6
2023-12-01 00:01:00,1809.75700,26.125362,25.588234,219.167450,42.049820,-25.824749,115.200000,-24.930000,-9.029999,130.829987,...,497.503540,165.913910,109.800000,-2.47698,-0.78408,13.609200,0.0,0.0,0.0,133.6
2023-12-01 00:02:00,1820.62354,26.125362,25.588234,222.390200,42.027847,-26.044480,114.500000,-24.680000,-9.029999,130.970000,...,496.771057,165.163269,109.800000,-2.47698,-0.78408,13.609200,0.0,0.0,0.0,133.6
2023-12-01 00:03:00,1813.32153,26.125362,25.588234,225.693512,41.522470,-25.517128,112.599991,-24.309998,-8.580000,130.989990,...,496.771057,165.163269,111.599991,-2.47698,-0.78408,13.609200,0.0,0.0,0.0,133.6
2023-12-01 00:04:00,1805.46729,26.125362,25.588234,240.034790,44.510796,-25.121613,114.000000,-24.610000,-8.330000,130.989990,...,496.771057,168.916443,111.599991,-2.47698,-0.78408,13.609200,0.0,0.0,0.0,133.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-07 23:56:00,2327.74658,28.176205,30.666515,316.736328,70.702590,-20.924774,102.300000,-21.680000,-9.059999,130.209991,...,501.165700,223.712860,307.800000,-7.09236,-1.08702,13.529999,0.0,0.0,0.0,131.0
2024-02-07 23:57:00,2326.25500,28.176205,30.666515,316.011230,70.702590,-20.924774,102.300000,-21.919998,-8.750000,130.209991,...,500.433300,224.463470,307.800000,-7.09236,-1.08702,13.529999,0.0,0.0,0.0,129.6
2024-02-07 23:58:00,2330.41968,28.176205,30.520023,316.011230,70.702590,-20.924774,102.800000,-22.129997,-8.410000,130.239990,...,500.433300,225.964752,307.800000,-7.09236,-1.08702,13.529999,0.0,0.0,0.0,129.6


In [27]:
df_nea.to_csv('colapsos_reducido_3.csv', index=True)